In [1]:
from pymatgen.ext.matproj import MPRester
from ase.data import chemical_symbols, atomic_numbers

from pymatgen.core.composition import Composition
from pymatgen.analysis.reaction_calculator import Reaction
from pymatgen.analysis.reaction_calculator import BalancedReaction

import numpy as np
import matplotlib.pyplot as plt
import pickle
from copy import deepcopy

# to use materials project stuff
# from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixEntry, MultiEntry, IonEntry

# to customize
from Packaged.pourbaix_final import PourbaixDiagram, PourbaixEntry, MultiEntry, IonEntry
from Packaged.MDFwithexperimental import ox_states, skip, only_solid, no_ions_for, overall_skipping
from Packaged.experimental_data import *

from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.core.ion import Ion
import itertools

In [2]:
# change these parameters in pourbaix_final to see changes
# # units: eV/V/atom
# FARADAYS = 1
# # units: K
# TEMP = 25 + 273
# # units: eV/K/atom
# R = 8.63 * (10**-5)
# RT = TEMP * R
# # units: eV
# H2O_gibbs = -2.4577835 
# LN_10 = np.log(10)

In [2]:
# using experimental / inputted data

# or define your own
# all_oxides = {"NiO": {"formation_energy_per_fu": -2.697,
#                             "inv_elements": ["O", "Ni"],
#                         },
#               "Ni3O4": {"formation_energy_per_fu": -6.953,
#                             "inv_elements": ["O", "Ni"],
#                         },
#               "Ni2O3": {"formation_energy_per_fu": -4.057,
#                             "inv_elements": ["O", "Ni"],
#                         },
#               "NiO2": {"formation_energy_per_fu": -1.343,
#                             "inv_elements": ["O", "Ni"],
#                         },
#               "Ni(OH)2": {"formation_energy_per_fu": -5.040,
#                             "inv_elements": ["O", "Ni"],
#                         },
#               "NiHO2": {"formation_energy_per_fu": -3.470,
#                             "inv_elements": ["O", "Ni"],
#                         }
#              }

In [8]:
# following definitions are potentially different from MDF with experimental notebook
def check_ox_state(entry):
    didnotcheck = []
    good = True
    for st in entry.name.split(" + "):
        if "(s)" in st:
            tempcomp = Composition(st.split("(s)")[0])
            elees = list(tempcomp.as_dict().keys())
            if "H" in elees or "O" in elees:
                guesses = tempcomp.oxi_state_guesses(oxi_states_override = ox_states, target_charge = 0)
                if not guesses:
                    print("OXIDATION STATE:", st, "is not valid")
                    good = False
                    break
            else:
                didnotcheck.append(st)
    return good, didnotcheck

def my_filter(entries, no_ions = False):
    solid_entries = [entry for entry in entries if entry.phase_type == "Solid"]
    sorted_entries = sorted(
        solid_entries,
        key=lambda x: (x.composition.reduced_composition, x.entry.energy_per_atom),
    )
    grouped_by_composition = itertools.groupby(sorted_entries, key=lambda x: x.composition.reduced_composition)
    min_entries = [list(grouped_entries)[0] for comp, grouped_entries in grouped_by_composition]
    if no_ions:
        all_entries = []
    else:
        all_entries = [entry for entry in entries if entry.phase_type == "Ion"]
    for en in min_entries:
        keep_going, unchecked = check_ox_state(en)
        if keep_going:
            eles = en.composition.as_dict().keys()
            if "H" in eles and "O" not in eles:
                continue
            all_entries.append(en)
    return all_entries 

def get_solids_in_sys(metals):
    solids = {}
    for ox in all_oxides.keys():
        for met in metals:
            if met in Composition(ox).as_dict().keys():
                solids[ox] = all_oxides[ox]
                break
    return solids

def get_ions_in_sys(metals):
    end_dict = {}
    for i in metals:
        if all_ions.get(i) is None:
            print("No ions found for:", i)
            return None
        end_dict.update(all_ions[i])
    return end_dict

def filter_by_ox_state(entries):
    uncheckeds = []
    new_entries = []
    for entry in entries:
        keep_going, unchecked = check_ox_state(entry)
        if len(entry.composition) != 1 and not keep_going:
            continue
        new_entries.append(entry)
        uncheckeds += unchecked
    print("Did not check the following:", list(set(uncheckeds)))
    return new_entries

def get_entries_with_my_filter(class_of_pourbaix, metals, con_dict, no_ions = False):
    mpr = MPRester("NGpMC4M2qzg2ZpPEX")
    if check_metals(metals):
        entries = get_pourbaix_entries(mpr, metals)
        filtered_ents = my_filter(entries, no_ions = no_ions)
        return filtered_ents
    return None

def gen_entries(metals, concentrations, MP_solids = False, MP_ions = False, ox_filter = True):
    if metals.sort() != list(concentrations.keys()).sort():
        print("Metals and concentrations do not match.")
        return
    if MP_solids:
        entries = get_entries_with_my_filter(PourbaixDiagram, METALS, {i: CONCEN for i in METALS}, no_ions = (not MP_ions))
        if not MP_ions:
            ions = get_ions_in_sys(metals)
            for io in ions.keys():
                if ions[io]["in_consideration"]:
                    entries.append(PourbaixEntry(IonEntry(Ion(ions[io]["without_charge"], charge = ions[io]["charge"]), ions[io]["energy"]), concentration = concentrations[ions[io]["major"]]))
        if ox_filter:
            entries = filter_by_ox_state(entries)
        return entries
    entries = []
    solids = get_solids_in_sys(metals)
    ions = get_ions_in_sys(metals)
    for so in solids.keys():
        # could end up adding custom correction
        entries.append(PourbaixEntry(ComputedEntry(so, solids[so]["formation_energy_per_fu"], correction = 0.0)))
    for io in ions.keys():
        if ions[io]["in_consideration"]:
            entries.append(PourbaixEntry(IonEntry(Ion(ions[io]["without_charge"], charge = ions[io]["charge"]), ions[io]["energy"]), concentration = concentrations[ions[io]["major"]]))
    for m in metals:
        entries.append(PourbaixEntry(ComputedEntry(m, 0, correction = 0.0)))
    return entries

In [9]:
def get_pourbaix_entries(self, chemsys, solid_compat="MaterialsProject2020Compatibility"):
    """
    A helper function to get all entries necessary to generate
    a Pourbaix diagram from the rest interface.

    Args:
        chemsys (str or [str]): Chemical system string comprising element
            symbols separated by dashes, e.g., "Li-Fe-O" or List of element
            symbols, e.g., ["Li", "Fe", "O"].
        solid_compat: Compatibility scheme used to pre-process solid DFT energies prior to applying aqueous
            energy adjustments. May be passed as a class (e.g. MaterialsProject2020Compatibility) or an instance
            (e.g., MaterialsProject2020Compatibility()). If None, solid DFT energies are used as-is.
            Default: MaterialsProject2020Compatibility
    """
    # imports are not top-level due to expense
    import itertools
    import warnings
    from pymatgen.core.periodic_table import Element
    from pymatgen.entries.computed_entries import ComputedEntry, ComputedStructureEntry

    from pymatgen.analysis.phase_diagram import PhaseDiagram
    from Packaged.pourbaix_final import PourbaixEntry, IonEntry
    from pymatgen.core.ion import Ion
    from pymatgen.entries.compatibility import (
        Compatibility,
        MaterialsProject2020Compatibility,
        MaterialsProjectAqueousCompatibility,
        MaterialsProjectCompatibility,
    )

    if solid_compat == "MaterialsProjectCompatibility":
        self.solid_compat = MaterialsProjectCompatibility()
    elif solid_compat == "MaterialsProject2020Compatibility":
        self.solid_compat = MaterialsProject2020Compatibility()
    elif isinstance(solid_compat, Compatibility):
        self.solid_compat = solid_compat
    else:
        raise ValueError(
            "Solid compatibility can only be 'MaterialsProjectCompatibility', "
            "'MaterialsProject2020Compatibility', or an instance of a Compatibility class"
        )

    pbx_entries = []

    if isinstance(chemsys, str):
        chemsys = chemsys.split("-")

    # Get ion entries first, because certain ions have reference
    # solids that aren't necessarily in the chemsys (Na2SO4)
    url = "/pourbaix_diagram/reference_data/" + "-".join(chemsys)
    ion_data = self._make_request(url)
    ion_ref_comps = [Composition(d["Reference Solid"]) for d in ion_data]
    ion_ref_elts = list(itertools.chain.from_iterable(i.elements for i in ion_ref_comps))
    ion_ref_entries = self.get_entries_in_chemsys(
        list(set([str(e) for e in ion_ref_elts] + ["O", "H"])),
        property_data=["e_above_hull"],
        compatible_only=False,
    )

    # suppress the warning about supplying the required energies; they will be calculated from the
    # entries we get from MPRester
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="You did not provide the required O2 and H2O energies.",
        )
        compat = MaterialsProjectAqueousCompatibility(solid_compat=self.solid_compat)
    # suppress the warning about missing oxidation states
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Failed to guess oxidation states.*")
        ion_ref_entries = compat.process_entries(ion_ref_entries)
    ion_ref_pd = PhaseDiagram(ion_ref_entries)

    # position the ion energies relative to most stable reference state
    for n, i_d in enumerate(ion_data):
        ion = Ion.from_formula(i_d["Name"])
        refs = [e for e in ion_ref_entries if e.composition.reduced_formula == i_d["Reference Solid"]]
        if not refs:
            raise ValueError("Reference solid not contained in entry list")
        stable_ref = sorted(refs, key=lambda x: x.data["e_above_hull"])[0]
        rf = stable_ref.composition.get_reduced_composition_and_factor()[1]

        solid_diff = ion_ref_pd.get_form_energy(stable_ref) - i_d["Reference solid energy"] * rf
        elt = i_d["Major_Elements"][0]
        correction_factor = ion.composition[elt] / stable_ref.composition[elt]
        energy = i_d["Energy"] + solid_diff * correction_factor
        ion_entry = IonEntry(ion, energy)
        pbx_entries.append(PourbaixEntry(ion_entry, f"ion-{n}"))

    # Construct the solid Pourbaix entries from filtered ion_ref entries

    extra_elts = set(ion_ref_elts) - {Element(s) for s in chemsys} - {Element("H"), Element("O")}
    for entry in ion_ref_entries:
        entry_elts = set(entry.composition.elements)
        # Ensure no OH chemsys or extraneous elements from ion references
        if not (entry_elts <= {Element("H"), Element("O")} or extra_elts.intersection(entry_elts)):
            # Create new computed entry
            form_e = ion_ref_pd.get_form_energy(entry)
            new_entry = ComputedEntry(entry.composition, form_e, entry_id=entry.entry_id)
            pbx_entry = PourbaixEntry(new_entry)
            pbx_entries.append(pbx_entry)

    return pbx_entries

In [10]:
def check_metals(metals, only_solid_fine = False):
    if not only_solid_fine:
        for metal in metals:
            if metal in (overall_skipping + only_solid):
                print("This metal cannot be included:", metal)
                return False
    else:
        for metal in metals:
            if metal in (overall_skipping):
                print("This metal cannot be included:", metal)
                return False
    return True

# separate after the HO part, if [ right after or ( right after, fine - otherwise, no transformation
# checking for HO2 (oxyhydroxide), (OH), and OH
def pretty_name(name):
    if name == "AlH3O3":
        return "Al2O3 * 3H2O"
    if "HO" not in name:
        return name
    if "HO2" in name:
        name = name.replace("HO2", "O(OH)")
    if "(HO)" in name:
        name = name.replace("(HO)", "(OH)")
    if "HO" in name:
        alls = name.split("HO")
        for i in range(1, len(alls)):
            if alls[i][0] == "(" or alls[i][0] == "[":
                alls[i] = "(OH)" + alls[i]
            else:
                alls[i] = "HO" + alls[i]
        name = ''.join(alls)
    return name

def find_ox_ion_pair(oxgrids, iongrids, minox, minion, index):
    for i, grid in enumerate(oxgrids):
        if grid[index] == minox[index]:
            ox_ind = i
            break
    for j, grid in enumerate(iongrids):
        if grid[index] == minion[index]:
            ion_ind = j
            break
    return (i, j)

def find_ox_ion_pairs(oxgrids, iongrids, minox, minion, indices):
    pairs = []
    for i in range(len(indices)):
        pairs.append(find_ox_ion_pair(oxgrids, iongrids, minox, minion, indices[i]))
    return pairs

def multiple_inds(difference, amt):
    flattened_ind = np.argsort(difference, axis = None)[:amt]
    inds = []
    for i in range(amt):
        inds.append(np.unravel_index(flattened_ind[i], difference.shape))
    return inds

def driving_force_2D(intermediates, metals, pH_range, potential_range, plot = True, zoom = 1):
    [oxide_grids, ion_grids, oxgrids, iongrids, oxides_inv, ions_inv, min_ox_ind, min_ox, min_ion, difference] = intermediates
    
    ox_to_mdf = {}
    for i, oxi in enumerate(oxide_grids.keys()):
        temp_diff = deepcopy(difference)
        temp_diff[min_ox_ind != i] = np.nan
        if np.isnan(temp_diff).all():
            print("There is no place in the given range where", oxi,  "is the most stable:")
            continue
        max_ind = np.unravel_index(np.nanargmin(temp_diff, axis=None), temp_diff.shape)
        (ox_ind, ion_ind) = find_ox_ion_pair(oxgrids, iongrids, min_ox, min_ion, max_ind)
        max_pH = round(pH_range[0][0] + max_ind[0]*pH_range[1], 5)
        max_poten = round(potential_range[0][0] + max_ind[1]*potential_range[1], 5)
        ox_to_mdf[oxi] = {"solid": pretty_name(oxides_inv[ox_ind]),
                          "ion": pretty_name(ions_inv[ion_ind]),
                          "pH": max_pH,
                          "potential": max_poten,
                          "diff (eV)": difference[max_ind]}
    
    max_ind = np.unravel_index(np.nanargmin(difference, axis=None), difference.shape)
    (ox_ind, ion_ind) = find_ox_ion_pair(oxgrids, iongrids, min_ox, min_ion, max_ind)
    max_pH = round(pH_range[0][0] + max_ind[0]*pH_range[1], 5)
    max_poten = round(potential_range[0][0] + max_ind[1]*potential_range[1], 5)
    
    if plot:
        # plot slices
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,6))
        # ax1: at the max's potential, ax2: at the max's pH, ax3: at middle potential
        pHr = np.arange((pH_range[0][0]), (pH_range[0][1]), pH_range[1])
        Ur = np.arange((potential_range[0][0]), (potential_range[0][1]), potential_range[1])
        len_pH = len(pHr)
        len_poten = len(Ur)
        midpoint = len_poten // 2
        mid_poten = round(potential_range[0][0] + midpoint*potential_range[1], 5)

        if len(metals) == 1:
            single = True
        else:
            single = False
            fig2, ax4 = plt.subplots(1, 1, figsize=(6,10))
            if zoom == 1:
                xaxis = pHr
                yind1 = np.arange(0, len_pH)
                yind2 = max_ind[1]
                xlab = "pH"
            elif zoom == 2:
                xaxis = Ur
                yind1 = max_ind[0]
                yind2 = np.arange(0, len_poten)
                xlab = "U (V)"
            elif zoom == 3:
                xaxis = pHr
                yind1 = np.arange(0, len_pH)
                yind2 = midpoint
                xlab = "pH"
            else:
                print("Will zoom in on plot 1 since no valid zoom number was given.")
                xaxis = pHr
                yind1 = np.arange(0, len_pH)
                yind2 = max_ind[1]
                xlab = "pH"

        for oxi in oxide_grids.keys():
            oxi_name = pretty_name(oxi)
            ax1.plot(pHr, oxide_grids[oxi][:, max_ind[1]], label = oxi_name)
            ax2.plot(Ur, oxide_grids[oxi][max_ind[0], :], label = oxi_name)
            ax3.plot(pHr, oxide_grids[oxi][:, midpoint], label = oxi_name)
            if not single:
                ax4.plot(xaxis, oxide_grids[oxi][yind1, yind2], label = oxi_name)
        for io in ion_grids.keys():
            ion_name = pretty_name(io)
            ax1.plot(pHr, ion_grids[io][:, max_ind[1]], '--', label = ion_name)
            ax2.plot(Ur, ion_grids[io][max_ind[0], :], '--', label = ion_name)
            ax3.plot(pHr, ion_grids[io][:, midpoint], '--', label = ion_name)
            if not single:
                ax4.plot(xaxis, ion_grids[io][yind1, yind2], '--', label = ion_name)
        ax1.set_title("U = " + str(max_poten) + "V")
        ax2.set_title("pH = " + str(max_pH))
        ax3.set_title("U = " + str(mid_poten) + "V")
        ax1.set_xlabel("pH")
        ax1.set_ylabel("Chemical Potential (eV)")
        ax2.set_xlabel("U (V)")
        ax3.set_xlabel("pH")
        handles, labels = ax3.get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(0.95, 1))
        if not single:
            ax4.set_title(("Plot " + str(zoom) + " zoomed in with mixed products plotted"))
            ax4.set_ylabel("Chemical Potential (eV)")
            ax4.set_xlabel(xlab)
            ax4.legend(loc='upper left', bbox_to_anchor=(1, 1))
    return ox_to_mdf

def print_dr_dict(max_dr):
    print(max_dr)
    plt.show()

In [11]:
def driving_force_2D_helper(species, metals, pH_range, potential_range, concen):
    len_pH = len(np.arange((pH_range[0][0]), (pH_range[0][1]), pH_range[1]))
    len_poten = len(np.arange((potential_range[0][0]), (potential_range[0][1]), potential_range[1]))
    pH, potential = np.mgrid[(pH_range[0][0]):(pH_range[0][1]):pH_range[1], potential_range[0][0]:potential_range[0][1]:potential_range[1]]
    
    ion_grids = {}
    oxide_grids = {}
        
    for entry in species:
        name = entry.name
        if entry.phase_type == "Solid":
            oxide_grids[name] = entry.normalized_energy_at_conditions(pH, potential)
        else:
            ion_grids[name] = entry.normalized_energy_at_conditions(pH, potential)
            
    oxgrids = np.array(list(oxide_grids.values()))
    iongrids = np.array(list(ion_grids.values()))
    oxides_inv = list(oxide_grids.keys())
    ions_inv = list(ion_grids.keys())
    if not oxides_inv:
        print("No solids left!")
        return
    if not ions_inv:
        print("No ions!")
        return
    min_ox_ind = np.argmin(oxgrids, axis = 0)
    min_ox = np.amin(oxgrids, axis = 0)
    min_ion = np.amin(iongrids, axis = 0)
    if np.shape(min_ox) != np.shape(oxgrids[0]) or np.shape(min_ox) != np.shape(min_ion) or np.shape(min_ox) != np.shape(min_ox_ind):
        print("Shapes do not match.")
        return
    difference = min_ox - min_ion
        
    return [oxide_grids, ion_grids, oxgrids, iongrids, oxides_inv, ions_inv, min_ox_ind, min_ox, min_ion, difference]

In [12]:
def sorted_top(max_dr):
    drs = []
    oxs = np.array(list(max_dr.keys()))
    for key in oxs:
        drs.append(max_dr[key]["diff (eV)"])
    ind = np.argsort(drs)
    return max_dr[oxs[ind[0]]]

In [7]:
# example using Materials Project data "experimental = False"
METALS = ["Cr", "Fe", "Ni"]
experimental = False
if check_metals(METALS):
    CONCEN = 1e-6
    CONCENTRATIONS = {i: CONCEN for i in METALS}
    precision = 0.01
    RANGE_PH = ([2.5, 12.5], precision)
    RANGE_POTENTIAL = ([-0.5, 0.75], precision)
    if experimental:
        SPECIES = gen_entries(METALS, CONCENTRATIONS)
    else:
        SPECIES = gen_entries(METALS, CONCENTRATIONS, MP_solids = True, MP_ions = True, ox_filter = True)
    intermediates = driving_force_2D_helper(SPECIES, METALS, RANGE_PH, RANGE_POTENTIAL, CONCEN)
    if intermediates:
        max_dr = driving_force_2D(intermediates, METALS, RANGE_PH, RANGE_POTENTIAL, plot = False)

  0%|          | 0/502 [00:00<?, ?it/s]

OXIDATION STATE: NiO3(s) is not valid
OXIDATION STATE: NiH(s) is not valid
OXIDATION STATE: Ni2O5(s) is not valid
OXIDATION STATE: Ni2H(s) is not valid
OXIDATION STATE: Ni3H(s) is not valid
OXIDATION STATE: Ni4O(s) is not valid
OXIDATION STATE: Ni4O3(s) is not valid
OXIDATION STATE: Ni5O4(s) is not valid
OXIDATION STATE: Ni5O11(s) is not valid
OXIDATION STATE: FeO2(s) is not valid
OXIDATION STATE: FeH(s) is not valid
OXIDATION STATE: Fe3O(s) is not valid
OXIDATION STATE: Fe3H(s) is not valid
OXIDATION STATE: Fe4O13(s) is not valid
OXIDATION STATE: CrH(s) is not valid
OXIDATION STATE: Cr2O(s) is not valid
OXIDATION STATE: Cr3O(s) is not valid
OXIDATION STATE: Cr4Fe2O19(s) is not valid
Did not check the following: ['Cr4Ni(s)', 'Cr3Fe(s)', 'Cr2Ni(s)', 'CrFe(s)', 'CrNi(s)', 'Cr2Fe(s)', 'Fe2Ni(s)', 'CrFe3(s)', 'FeNi(s)', 'Cr3Ni(s)', 'FeNi3(s)', 'FeNi2(s)', 'Fe3Ni2(s)', 'Cr(s)', 'Fe(s)', 'CrNi2(s)', 'CrFe4(s)', 'Ni(s)', 'CrNi3(s)', 'Fe3Ni(s)']
There is no place in the given range where Ni(s)

In [13]:
# Materials Project - max_dr gives all solids with DFs < 0
max_dr

{'CrO2(s)': {'solid': 'CrO2(s)',
  'ion': 'CrO4[2-]',
  'pH': 7.11,
  'potential': 0.74,
  'diff (eV)': 0.6231919282031138},
 'Cr2O3(s)': {'solid': 'Cr2O3(s)',
  'ion': 'Cr(OH)[2+]',
  'pH': 12.49,
  'potential': -0.47,
  'diff (eV)': -0.8864344210974704},
 'Cr5O12(s)': {'solid': 'Cr5O12(s)',
  'ion': 'CrO4[2-]',
  'pH': 9.11,
  'potential': 0.74,
  'diff (eV)': 1.0968510015349606}}

In [14]:
# Materials Project - sorted_top will give best MDF 
sorted_top(max_dr)

{'solid': 'Cr2O3(s)',
 'ion': 'Cr(OH)[2+]',
 'pH': 12.49,
 'potential': -0.47,
 'diff (eV)': -0.8864344210974704}

In [15]:
# example using Materials Project data "experimental = True"
METALS = ["Cr", "Fe", "Ni"]
experimental = True # everything else other than this line is the same as previous run
if check_metals(METALS):
    CONCEN = 1e-6
    CONCENTRATIONS = {i: CONCEN for i in METALS}
    precision = 0.01
    RANGE_PH = ([2.5, 12.5], precision)
    RANGE_POTENTIAL = ([-0.5, 0.75], precision)
    if experimental:
        SPECIES = gen_entries(METALS, CONCENTRATIONS)
    else:
        SPECIES = gen_entries(METALS, CONCENTRATIONS, MP_solids = True, MP_ions = True, ox_filter = True)
    intermediates = driving_force_2D_helper(SPECIES, METALS, RANGE_PH, RANGE_POTENTIAL, CONCEN)
    if intermediates:
        max_dr = driving_force_2D(intermediates, METALS, RANGE_PH, RANGE_POTENTIAL, plot = False)

There is no place in the given range where CrO(s) is the most stable:
There is no place in the given range where Cr(HO)2(s) is the most stable:
There is no place in the given range where FeO(s) is the most stable:
There is no place in the given range where Fe(HO)2(s) is the most stable:
There is no place in the given range where Fe3O4(s) is the most stable:
There is no place in the given range where Fe2O3(s) is the most stable:
There is no place in the given range where Ni(HO)2(s) is the most stable:
There is no place in the given range where NiO(s) is the most stable:
There is no place in the given range where Ni3O4(s) is the most stable:
There is no place in the given range where Ni2O3(s) is the most stable:
There is no place in the given range where NiO2(s) is the most stable:
There is no place in the given range where Cr(s) is the most stable:
There is no place in the given range where Fe(s) is the most stable:
There is no place in the given range where Ni(s) is the most stable:


In [16]:
# using experimental data - max_dr gives all solids with DFs < 0
max_dr

{'Cr2O3(s)': {'solid': 'Cr2O3(s)',
  'ion': 'CrO2[-]',
  'pH': 8.47,
  'potential': -0.26,
  'diff (eV)': -0.46751946793292376},
 'CrO2(s)': {'solid': 'CrO2(s)',
  'ion': 'CrO4[2-]',
  'pH': 9.27,
  'potential': 0.74,
  'diff (eV)': 1.1619009005839436},
 'CrO3(s)': {'solid': 'CrO3(s)',
  'ion': 'CrO4[2-]',
  'pH': 11.55,
  'potential': 0.74,
  'diff (eV)': 1.7018377044334443}}

In [17]:
# experimental data
sorted_top(max_dr)

{'solid': 'Cr2O3(s)',
 'ion': 'CrO2[-]',
 'pH': 8.47,
 'potential': -0.26,
 'diff (eV)': -0.46751946793292376}

In [15]:
# Note that for example even Fe(s) differs because 
# 1) there is inherent difference in Fe[2+]'s value (about 0.07)
# 2) MP does ion corrections* based on the reference solid, which contributes the other about 0.04
# *we currently don't do ion corrections at all